# Imports

In [179]:
import numpy as np
import matplotlib.pyplot as plt

import json # original json library

import pandas as pd
import logging

## Log set-up

In [180]:
LOG_FILENAME = '/home/benjamin/Folders_Python/Cyber/logs/logfile.log'
LOG_FORMAT = '%(asctime)% -- %(name)s -- %(levelname)s -- %(message)s'
# LOG_LEVEL = logging.INFO

In [181]:
# specific logger for the module
logger = logging.getLogger(__name__)   # creates specific logger for the module
logger.setLevel(logging.DEBUG)    # entry level of messages from all handlers
LOG_FORMAT = '%(asctime)s -- %(name)s -- %(levelname)s -- %(message)s'
formatter = logging.Formatter(LOG_FORMAT)

# file handler to log everything
file_handler = logging.FileHandler(LOG_FILENAME, mode='w')
file_handler.setLevel(logging.INFO)  # all messages (DEBUG and up) get logged in the file
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# stream handler to show messages to the console
console = logging.StreamHandler()
console.setLevel(logging.WARNING)  # Warning messages and up get displayed to the console
console.setFormatter(formatter)
logger.addHandler(console)

# start your engine
logger.info("-------- new run --------")

# Import pcap file

In [182]:
# NB : tshark -r <file>.pcap -T json > <file_pcap>.json -t r
# commande shell qui prend un pcap et le passe en json

!rm /home/benjamin/Folders_Python/Cyber/data/outputs/input_pcap.json
!tshark -r /home/benjamin/Folders_Python/Cyber/data/input_pcaps/input.pcap -T json -t r > /home/benjamin/Folders_Python/Cyber/data/outputs/input_pcap.json

logger.info("run tshark to create json translation of input.pcap")

In [183]:
class Packet():
    """Utility self-made unperfect class to parse the json object and extract features from a packet-like dict
    """
    
    def __init__(self, raw_packet:dict) -> None:
        self.raw_packet = raw_packet
        self._packet_data = None
        # logger.debug('constructor of Packet instance has finished')
        
    @property
    def packet_data(self):
        # returns the full dictionnary of features
        if self._packet_data is not None:
            return self._packet_data
        else:
            sl = self.raw_packet.get('_source').get('layers')
            slf = sl.get('frame')
            sle = sl.get('eth')
            sli = sl.get('ip', {})  # return empty dict as default not found value so it can handle another get method
            slu = sl.get('udp', {})
            slt = sl.get('tcp', {})
                                               
            self._packet_data = {
                'frame_time' : slf.get('frame.time'),
                'frame_time_relative' : slf.get('frame.time_relative'),
                'frame_length' : slf.get("frame.len"),
                'frame_protocols' : slf.get("frame.protocols"),
                'eth_source': sle.get("eth.src"),
                'eth_dest': sle.get("eth.dst") ,
                'ip_version': sli.get("ip.version"),
                'ip_header_length': sli.get("ip.hdr_len"),
                'ip_length': sli.get("ip.len"),
                'ip_id': sli.get("ip.id"),
                'ip_flags': sli.get("ip.flags"),
                'ip_ttl': sli.get("ip.ttl"),
                'ip_proto': sli.get("ip.proto"),
                'ip_source': sli.get("ip.src"),
                'ip_dest': sli.get("ip.dst"),
                'udp_source_port': slu.get("udp.srcport"),
                'udp_dest_port': slu.get("udp.port"),
                'udp_length': slu.get("udp.length"),
                'tcp_source_port': slt.get("tcp.srcport"),
                'tcp_dest_port': slt.get("tcp.dstport"),
                'tcp_length': slt.get("tcp.len"),
                'tcp_flags': slt.get("tcp.flags"),
            }
            # logger.debug('packet_data @property method has finished')
            return self._packet_data
        
    @packet_data.setter
    def packet_data(self, input):
        """illegal attempt to write packet_data"""
        logger.warning('Illegal attempt to write a data_packet in a packet object')
        pass

In [184]:
PCAP_FILENAME = "/home/benjamin/Folders_Python/Cyber/data/outputs/input_pcap.json"

with open (PCAP_FILENAME, errors='replace') as raw_packets:  # NB : errors='replace' bypasses decoding errors
    json_object = json.load(raw_packets)    # load le fichier json dans une structure Python (list of dicts)

In [185]:
#Exemple : premier dict de la liste : c'est un paquet (=une frame Ethernet)

json_object[0]

{'_index': 'packets-2011-01-25',
 '_type': 'doc',
 '_score': None,
 '_source': {'layers': {'frame': {'frame.encap_type': '1',
    'frame.time': 'Jan 25, 2011 19:52:22.484409000 CET',
    'frame.offset_shift': '0.000000000',
    'frame.time_epoch': '1295981542.484409000',
    'frame.time_delta': '0.000000000',
    'frame.time_delta_displayed': '0.000000000',
    'frame.time_relative': '0.000000000',
    'frame.number': '1',
    'frame.len': '997',
    'frame.cap_len': '997',
    'frame.marked': '0',
    'frame.ignored': '0',
    'frame.protocols': 'eth:ethertype:ip:tcp:http'},
   'eth': {'eth.dst': '00:1a:8c:15:f9:80',
    'eth.dst_tree': {'eth.dst_resolved': '00:1a:8c:15:f9:80',
     'eth.dst.oui': '6796',
     'eth.dst.oui_resolved': 'Sophos Ltd',
     'eth.addr': '00:1a:8c:15:f9:80',
     'eth.addr_resolved': '00:1a:8c:15:f9:80',
     'eth.addr.oui': '6796',
     'eth.addr.oui_resolved': 'Sophos Ltd',
     'eth.dst.lg': '0',
     'eth.lg': '0',
     'eth.dst.ig': '0',
     'eth.ig': 

In [186]:
# exemple d'instanciation d'un objet Packet
p = Packet(json_object[0])

p.packet_data

{'frame_time': 'Jan 25, 2011 19:52:22.484409000 CET',
 'frame_time_relative': '0.000000000',
 'frame_length': '997',
 'frame_protocols': 'eth:ethertype:ip:tcp:http',
 'eth_source': '40:61:86:9a:f1:f5',
 'eth_dest': '00:1a:8c:15:f9:80',
 'ip_version': '4',
 'ip_header_length': '20',
 'ip_length': '983',
 'ip_id': '0x000076e0',
 'ip_flags': '0x00000040',
 'ip_ttl': '128',
 'ip_proto': '6',
 'ip_source': '192.168.3.131',
 'ip_dest': '72.14.213.138',
 'udp_source_port': None,
 'udp_dest_port': None,
 'udp_length': None,
 'tcp_source_port': '57011',
 'tcp_dest_port': '80',
 'tcp_length': '943',
 'tcp_flags': '0x00000018'}

# Produce DataFrame for Raw Packets analysis

In [187]:
# créé la liste de dictionnaires des data des objets Packets
packets = [ Packet(d).packet_data for d in json_object ]

In [188]:
df_packets = pd.DataFrame(packets)

In [189]:
df_packets

,frame_time,frame_time_relative,frame_length,frame_protocols,eth_source,eth_dest,ip_version,ip_header_length,ip_length,ip_id,...,ip_proto,ip_source,ip_dest,udp_source_port,udp_dest_port,udp_length,tcp_source_port,tcp_dest_port,tcp_length,tcp_flags
0,"Jan 25, 2011 19:52:22.484409000 CET",0.000000000,997,eth:ethertype:ip:tcp:http,40:61:86:9a:f1:f5,00:1a:8c:15:f9:80,4,20,983,0x000076e0,...,6,192.168.3.131,72.14.213.138,None,None,None,57011,80,943,0x00000018
1,"Jan 25, 2011 19:52:22.514250000 CET",0.029841000,440,eth:ethertype:ip:tcp:http:data-text-lines,00:1a:8c:15:f9:80,40:61:86:9a:f1:f5,4,20,426,0x00002d62,...,6,72.14.213.138,192.168.3.131,None,None,None,80,57011,386,0x00000018
2,"Jan 25, 2011 19:52:22.708292000 CET",0.223883000,66,eth:ethertype:ip:tcp,40:61:86:9a:f1:f5,00:1a:8c:15:f9:80,4,20,52,0x00002e9e,...,6,192.168.3.131,72.14.213.102,None,None,None,55950,80,0,0x00000002
3,"Jan 25, 2011 19:52:22.713832000 CET",0.229423000,54,eth:ethertype:ip:tcp,40:61:86:9a:f1:f5,00:1a:8c:15:f9:80,4,20,40,0x000076ef,...,6,192.168.3.131,72.14.213.138,None,None,None,57011,80,0,0x00000010
4,"Jan 25, 2011 19:52:22.727058000 CET",0.242649000,66,eth:ethertype:ip:tcp,00:1a:8c:15:f9:80,40:61:86:9a:f1:f5,4,20,52,0x00009415,...,6,72.14.213.102,192.168.3.131,None,None,None,80,55950,0,0x00000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14256,"Jan 25, 2011 19:57:20.768972000 CET",298.284563000,54,eth:ethertype:ip:tcp,08:00:27:cc:3f:1b,52:54:00:12:35:02,4,20,40,0x000088b3,...,6,10.0.2.15,65.54.167.27,None,None,None,2548,5480,0,0x00000014
14257,"Jan 25, 2011 19:57:20.787428000 CET",298.303019000,162,eth:ethertype:ip:udp:db-lsp-disc:json,40:61:86:9a:f1:f5,ff:ff:ff:ff:ff:ff,4,20,148,0x000035d7,...,17,192.168.3.131,255.255.255.255,17500,17500,128,None,None,None,None
14258,"Jan 25, 2011 19:57:20.789753000 CET",298.305344000,162,eth:ethertype:ip:udp:db-lsp-disc:json,40:61:86:9a:f1:f5,ff:ff:ff:ff:ff:ff,4,20,148,0x000035d8,...,17,192.168.3.131,192.168.3.255,17500,17500,128,None,None,None,None
14259,"Jan 25, 2011 19:57:20.987428000 CET",298.503019000,162,eth:ethertype:ip:udp:db-lsp-disc:json,40:61:86:9a:f1:f5,ff:ff:ff:ff:ff:ff,4,20,148,0x000035d7,...,17,192.168.3.131,255.255.255.255,17500,17500,128,None,None,None,None


In [190]:
df_packets.describe(include='all').transpose()

,count,unique,top,freq
frame_time,14261,14123,"Jan 25, 2011 19:57:15.280863000 CET",4
frame_time_relative,14261,14123,292.796454000,4
frame_length,14261,857,1514,4378
frame_protocols,14261,44,eth:ethertype:ip:tcp,10315
eth_source,14261,8,00:1a:8c:15:f9:80,6170
eth_dest,14261,12,40:61:86:9a:f1:f5,6184
ip_version,14243,1,4,14243
ip_header_length,14243,1,20,14243
ip_length,14243,858,40,5319
ip_id,14243,11304,0x00000000,79


# EVE JSON Output by Suricata

In [191]:
# run Suricata to produce an eve.json file with alerts

!rm /home/benjamin/Folders_Python/Cyber/data/outputs/eve.json
!suricata -r /home/benjamin/Folders_Python/Cyber/data/input_pcaps/input.pcap -l /home/benjamin/Folders_Python/Cyber/data/outputs -k none

logger.info("run Suricata to reassemble flows and create alert logs")

22/6/2023 -- 23:36:07 - <Info> - Configuration node 'af-packet' redefined.
22/6/2023 -- 23:36:07 - <Notice> - This is Suricata version 6.0.1 RELEASE running in USER mode
22/6/2023 -- 23:37:13 - <Warning> - [ERRCODE: SC_ERR_INITIALIZATION(45)] - Unix socket: UNIX socket bind(/var/run/suricata-command.socket) error: Address already in use
22/6/2023 -- 23:37:13 - <Warning> - [ERRCODE: SC_ERR_INITIALIZATION(45)] - Unable to create unix command socket
22/6/2023 -- 23:37:13 - <Notice> - all 5 packet processing threads, 4 management threads initialized, engine started.
22/6/2023 -- 23:37:13 - <Notice> - Signal Received.  Stopping engine.
22/6/2023 -- 23:37:13 - <Notice> - Pcap-file module read 1 files, 14261 packets, 9216531 bytes


In [192]:
# Pandas provides a useful method – json_normalize – for normalizing nested JSON fields into dataframe. Resulting columns use dot notation to signify nested objects, similar to how Elasticsearch does it

SURICATA_EVE_LOG = "/home/benjamin/Folders_Python/Cyber/data/outputs/eve.json"

with open (SURICATA_EVE_LOG) as packets:
    df_log = pd.json_normalize(
        [json.loads(packet) for packet in packets],
        max_level=1
    )

In [193]:
df_log

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,community_id,...,stats.uptime,stats.decoder,stats.flow,stats.defrag,stats.flow_bypassed,stats.tcp,stats.detect,stats.app_layer,stats.http,stats.ftp
0,2011-01-25T19:52:22.968559+0100,3.905032e+14,20.0,tls,192.168.3.131,52152.0,72.14.213.147,443.0,TCP,1:F2Sb+8lkB1KXk6ssGMWJxClEWVc=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-25T19:52:23.874614+0100,5.980360e+14,169.0,http,192.168.3.131,55958.0,65.55.239.163,80.0,TCP,1:NNi9qcb4aCsmruzKaTwr7mOohqg=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-25T19:52:23.874614+0100,5.980360e+14,169.0,fileinfo,65.55.239.163,80.0,192.168.3.131,55958.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01-25T19:52:23.881606+0100,9.969087e+12,170.0,http,192.168.3.131,55960.0,206.108.207.139,80.0,TCP,1:uv3SvM97EooRFrkLLIYb/tnn+ko=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-01-25T19:52:23.881606+0100,9.969087e+12,170.0,fileinfo,206.108.207.139,80.0,192.168.3.131,55960.0,TCP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,2011-01-25T19:52:22.484409+0100,2.110799e+15,NaN,flow,10.0.2.15,2549.0,65.55.116.184,5480.0,TCP,1:vdGgmT4V4n0ffNp8Z2O/SAhtx2g=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028,2011-01-25T19:52:22.484409+0100,1.970154e+15,NaN,flow,10.0.2.15,62310.0,10.0.2.3,53.0,UDP,1:ZLDD2UK8/D8Lo6bUXIIsDHijWIU=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029,2011-01-25T19:52:22.484409+0100,2.814389e+14,NaN,flow,192.168.3.131,57949.0,195.27.155.40,80.0,TCP,1:d6pVTFel5IMpeS/5fyyH9OlKwYI=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2030,2011-01-25T19:52:22.484409+0100,1.407373e+15,NaN,flow,172.16.255.1,50983.0,88.203.161.148,13899.0,UDP,1:Y9vY0g/fiZ04Jp13fn/+eLmLSic=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
df_log[df_log['event_type']=='flow']

,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,community_id,...,stats.uptime,stats.decoder,stats.flow,stats.defrag,stats.flow_bypassed,stats.tcp,stats.detect,stats.app_layer,stats.http,stats.ftp
993,2011-01-25T19:52:22.484409+0100,1.738788e+15,NaN,flow,172.16.255.1,137.0,172.16.255.255,137.0,UDP,1:3O12rzx4M+K4kYX08vSUdN3p2TQ=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,2011-01-25T19:52:22.484409+0100,1.057426e+15,NaN,flow,10.0.2.15,48575.0,10.0.2.2,1900.0,UDP,1:aRVJ4wV3qBwT/3ze/Eo0oNQmo04=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021,2011-01-25T19:52:22.484409+0100,1.193001e+15,NaN,flow,192.168.3.131,57011.0,72.14.213.138,80.0,TCP,1:QkIgN7TrgAaMek+fYbCecSXUZc8=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1027,2011-01-25T19:52:22.484409+0100,1.738788e+15,NaN,flow,172.16.255.1,137.0,172.16.255.255,137.0,UDP,1:3O12rzx4M+K4kYX08vSUdN3p2TQ=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1031,2011-01-25T19:52:22.484409+0100,1.660353e+15,NaN,flow,192.168.3.131,56050.0,66.235.139.121,80.0,TCP,1:uWVO8VoQ6jwKWxGMyFOOvraIWA0=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,2011-01-25T19:52:22.484409+0100,2.806852e+14,NaN,flow,172.16.255.1,10617.0,204.9.163.158,80.0,TCP,1:ob9Wwm8p381An7MS3syuBwJZWa0=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027,2011-01-25T19:52:22.484409+0100,2.110799e+15,NaN,flow,10.0.2.15,2549.0,65.55.116.184,5480.0,TCP,1:vdGgmT4V4n0ffNp8Z2O/SAhtx2g=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028,2011-01-25T19:52:22.484409+0100,1.970154e+15,NaN,flow,10.0.2.15,62310.0,10.0.2.3,53.0,UDP,1:ZLDD2UK8/D8Lo6bUXIIsDHijWIU=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029,2011-01-25T19:52:22.484409+0100,2.814389e+14,NaN,flow,192.168.3.131,57949.0,195.27.155.40,80.0,TCP,1:d6pVTFel5IMpeS/5fyyH9OlKwYI=,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Produce DatFrame for Flow Analysis

In [195]:
# from Suricata doc :

# 15.1.2.12. Event type: Flow
# 15.1.2.12.1. Fields

#     “pkts_toserver”: total number of packets to server, include bypassed packets
#     “pkts_toclient”: total number of packets to client
#     “bytes_toserver”: total bytes count to server
#     “bytes_toclient”: total bytes count to client
#     “bypassed.pkts_toserver”: number of bypassed packets to server
#     “bypassed.pkts_toclient”: number of bypassed packets to client
#     “bypassed.bytes_toserver”: bypassed bytes count to server
#     “bypassed.bytes_toclient”: bypassed bytes count to client
#     “start”: date of start of the flow
#     “end”: date of end of flow (last seen packet)
#     “age”: duration of the flow
#     “bypass”: if the flow has been bypassed, it is set to “local” (internal bypass) or “capture”
#     “state”: display state of the flow (include “new”, “established”, “closed”, “bypassed”)
#     “reason”: mechanism that did trigger the end of the flow (include “timeout”, “forced” and “shutdown”)
#     “alerted”: “true” or “false” depending if an alert has been seen on flow

In [196]:
# https://www.stamus-networks.com/blog/jupyter-playbooks-for-suricata-part-1

# https://malware-traffic-analysis.net/

In [197]:
class Flow():
    """Utility class - takes a event-flow string out of eve.json,
       creates a one-level dict structure, suitable for dataframe creation
    """
    def __init__(self, flow_event:dict):
        if flow_event.get('event_type') != 'flow':
            logger.critical("Attempt to build a Flow instance with a non-flow event")
            raise ValueError
        self._raw_flow_event = flow_event
        self._features = None
        
    @property
    def features(self):
        if self._features is not None:
            return self._features
        else:
            keys_list_first_level = [
                'timestamp',
                'flow_id',
                'src_ip',
                'src_port',
                'dest_ip',
                'dest_port',
                'proto'
            ]
            keys_list_second_level = [
                'pkts_toserver',
                'pkts_toclient',
                'bytes_toserver',
                'bytes_toclient',
                'start',
                'end',
                'age',
                'state',
                'reason',
                'alerted'
            ]
            d1 = { k: self._raw_flow_event.get(k) for k in keys_list_first_level }
            d2 = { k: self._raw_flow_event.get('flow').get(k) for k in keys_list_second_level }
            self._features = { **d1, **d2 }
            # logger.info("built a Flow features object")
            return self._features
        
    @features.setter
    def features(self, input):
        logger.critical("illegal attempt to hard write features in a Flow object")
        
    def __str__(self) -> str:
        return json.dumps(self.features, indent=4)
    
    def __repr__(self) -> str:
        return json.dumps(self.features, indent=4)


In [198]:
# exemple de flow JSON = 
# {
# "timestamp":"2023-06-17T10:46:05.765744+0200",
# "flow_id":860724109937755,
# "event_type":"flow",
# "src_ip":"2a01:cb19:872e:3000:0e4f:3187:540c:d66c",
# "src_port":47864,
# "dest_ip":"2a00:1450:4007:081a:0000:0000:0000:2003",
# "dest_port":80,
# "proto":"TCP",
# "flow":
#     {"pkts_toserver":6,
#     "pkts_toclient":5,
#     "bytes_toserver":516,
#     "bytes_toclient":430,
#     "start":"2023-06-17T10:46:10.625755+0200",
#     "end":"2023-06-17T10:46:44.150502+0200",
#     "age":34,
#     "state":"new",
#     "reason":"shutdown",
#     "alerted":true},
# "community_id":"1:uRhWV544zvWeIohZCmryZHXZ5EA=",
# "tcp":
#     {"tcp_flags":"00",
#     "tcp_flags_ts":"00",
#     "tcp_flags_tc":"00"
#     }
# }'

In [199]:
SURICATA_EVE_LOG = "/home/benjamin/Folders_Python/Cyber/data/outputs/eve.json"

i=0
columns_names =  [
                'timestamp',
                'flow_id',
                'src_ip',
                'src_port',
                'dest_ip',
                'dest_port',
                'proto'
            ] + [
                'pkts_toserver',
                'pkts_toclient',
                'bytes_toserver',
                'bytes_toclient',
                'start',
                'end',
                'age',
                'state',
                'reason',
                'alerted'
            ]
dict_for_dataframe = { k:[] for k in columns_names }

with open (SURICATA_EVE_LOG) as f:
    for event_string in f:
        python_object = json.loads(event_string)
        if python_object.get('event_type')=='flow':
            flow = Flow(python_object)
            for k in columns_names:
                if dict_for_dataframe[k] == []:
                    dict_for_dataframe[k] = [flow.features.get(k)]
                else:
                    dict_for_dataframe[k].append(flow.features.get(k))

In [200]:
df_flow = pd.DataFrame(data=dict_for_dataframe)

In [201]:
df_flow

,timestamp,flow_id,src_ip,src_port,dest_ip,dest_port,proto,pkts_toserver,pkts_toclient,bytes_toserver,bytes_toclient,start,end,age,state,reason,alerted
0,2011-01-25T19:52:22.484409+0100,1738787919544930,172.16.255.1,137.0,172.16.255.255,137.0,UDP,6,0,552,0,2011-01-25T19:54:00.639586+0100,2011-01-25T19:54:17.717731+0100,17,new,timeout,False
1,2011-01-25T19:52:22.484409+0100,1057425718997994,10.0.2.15,48575.0,10.0.2.2,1900.0,UDP,10,0,1745,0,2011-01-25T19:54:19.160746+0100,2011-01-25T19:54:21.186212+0100,2,new,timeout,False
2,2011-01-25T19:52:22.484409+0100,1193000648991801,192.168.3.131,57011.0,72.14.213.138,80.0,TCP,6,3,3163,1496,2011-01-25T19:52:22.484409+0100,2011-01-25T19:52:24.437888+0100,2,new,timeout,False
3,2011-01-25T19:52:22.484409+0100,1738787927912513,172.16.255.1,137.0,172.16.255.255,137.0,UDP,3,0,276,0,2011-01-25T19:56:08.946241+0100,2011-01-25T19:56:10.474855+0100,2,new,timeout,False
4,2011-01-25T19:52:22.484409+0100,1660353222151205,192.168.3.131,56050.0,66.235.139.121,80.0,TCP,2,2,120,126,2011-01-25T19:52:50.986149+0100,2011-01-25T19:53:03.395785+0100,13,closed,timeout,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,2011-01-25T19:52:22.484409+0100,280685176902890,172.16.255.1,10617.0,204.9.163.158,80.0,TCP,5,5,347,581,2011-01-25T19:53:55.087274+0100,2011-01-25T19:53:55.336011+0100,0,closed,shutdown,False
637,2011-01-25T19:52:22.484409+0100,2110798668600534,10.0.2.15,2549.0,65.55.116.184,5480.0,TCP,5,5,788,718,2011-01-25T19:56:21.169174+0100,2011-01-25T19:56:21.390554+0100,0,closed,shutdown,False
638,2011-01-25T19:52:22.484409+0100,1970153514340278,10.0.2.15,62310.0,10.0.2.3,53.0,UDP,2,2,148,320,2011-01-25T19:54:24.266166+0100,2011-01-25T19:56:14.272312+0100,110,established,shutdown,False
639,2011-01-25T19:52:22.484409+0100,281438949311109,192.168.3.131,57949.0,195.27.155.40,80.0,TCP,2,1,108,60,2011-01-25T19:55:21.426629+0100,2011-01-25T19:55:21.612810+0100,0,new,shutdown,False
